## Locks

Mainly used to ensure execution of code as if it were a single atomic instruction

### 28.1 Basic Idea

A lock is just a variable that must be declared as a type lock_t (may change based on library). This lock variable holds the state of the lock at any instant in time. It is either available/unlocked/free and no thread holds the lock or acquired and exactly one thread holds the lock.  
Lock and unlock are pretty simple, calling lock() tried to acquire the lock if the lock is free and the lock will acquire the lock and enter critical section. This thread is said to be the owner of the lock. If another thread calls lock on the same variable, it will not return while the lock is held by another thread. This way, other threads are prevented from entering the critical execution section while the first thread is there.  
Once the owner calls unlock, the lock is now free. If no other threads are waiting for the lock, the lock is simply changed to free. If there are waiting threads, one of them will notice the change in the state of the lock, acquire the lock and enter the critical section  
Threads are entities created by the programmer, but scheduled by the OS. Locks help transform the chaos that traditional OS scheduling is into a more controlled activity.

### 28.2 Pthread Locks

The POSIX library names locks a mutex, as it is used to provide mutual exclusion between threads, i.e. if a thread is in the critical section, it excludes the others from entering until it has completed the critical section.  
The POSIX implementation passes a variable to lock and unlock as there may be different locks to protect different variables. This increases concurrency. Instead of one big lock used any time any critical section is accessed, one will protect different data with different locks, allowing more threads to be in locked code at once.

### 28.3 Building a Lock

To build a lock, we need to use the OS and the hardware.

### 28.4 Evaluating Locks

Goals of Locks:
1. Mutual Exclusion: Does the lock work preventing multiple threads from entering critical section
2. Fairness: Does each thread vying for a lock get a fair shot at acquiring it when it is free, and does any lock starve
3. Performance: Time overhead added by using the lock

### 28.5 Controlling Interrupts

One of the earliest solutions for handling interrupts was to simply disable them at lock and reenable them after unlock.  
This is easy to do, but has a lot of drawbacks. This requires a priveleged operation, and trust that this is not abused. A greedy processor could hog the entire processor. 
This also doesnt work on multiprocessors since many threads could be running on different CPUs and each could enter the critical section.  
Turning off interrupts can lead to interrupts being lost, which can cause system problems  
This is also pretty inefficient. Turning off and turning on signals can be very slow.

### 28.6 Using Loads/Stores

The idea is to use a flag to indicate wether a thread has possession of a lock.  
The first thread that enters the critical section will call lock() which tests whether the flag is 1 and then sets the flag to 1 to indicate that the thread now holds the lock. When finished, the thread calls unlock and clears the flag.  
If a thread calls lock when the first thread is in the critical section, it will spin wait in the while look for that thread to call unlock and clear the flag. Once the first thread does so, the waiting thread will go into the critical section and set the flag to 1 like the previous section.  
The code has two problems, correctness and performance.  
1. Correctness: What happens when you interrupt and switch threads right as you set the flag?
2. Performance: Spin waiting wastes time waiting for anothe thread to release a lock, and is super wasteful

### 28.7 Building Working Spin Locks with test-and-set

Because flags don't work and load/store doesn't work, specialized hardware had to be developed. The simplest bit of hardware support is the test-and-set instruction. It returns the old value and updates the balue to the new value. It allows yo uto test the old balue and set the new value and allows us to build a spin lock. To work properly, it requires a preemptive schedule. Without preemption, a thread will never relinquish the lock.

### 28.8 Evaluating Spin Locks

The spin lock only allows a single thread to enter the critical section, so it is correct.  
However, spin locks don't provide fairness guarantees. A thread could spin forever which may lead to starvation.  
Spin locks are very wasteful in single processors, but on multiplle cpus, it is quite effective.

### 28.9 Compare-And-Swap

Another primitive that is provided is the compare and swap or compare and exchange. The idea is to test whether the value at the address in pointer is equal to the expected value and update with the new value if it is not.  
The way you implement this is to call the compare and swap method in your while loop similar to the test and set.

### 28.10 Load Linked and Store Conditional

The load-linked and store conditional instructions can be used to build locks and other structures. It's similar to compare and swap, but that it returns 0 and doesn't update the pointer if it returns.

### 28.11-28.12 other random hardare instructions

### 28.13 Just yield- Simple approach

What do we do if a context switch occurs in a critical section, and threads start to spin, waiting for the interrupted thread to be run?  
You can just yield.  
A thread can call yield whenever it wants to give up the CPU. It moves the caller from the running state into the ready state and promotes another thread to running.  
This appeoach doesn't tackle starvation or the overhead of switches.

### 28.14 Using Queues: Sleeping Instead of Spinning

The problem with our previous approaches is that we don't control enough and we hope that our threads will run to completion. The scheduler could make a bad choice, if the thread will yield the CPU immediately or spin waiting for the lock.  
If we use a queue keeping all the threads waiting for the lock, we will be able to control which thread will acquire the lock next.  
We can use park to put a calling thread to sleep, unpark to wake up a thread. These two routines can be used to build a lock that puts a caller to sleep if it tries to acquire a held lock and wakes it up when the lock is free.  
We can use test and set and a queue. These two ideas will allow control over who gets the lock and avoid starvation. This approach should minimize spinning time, as a thread may be interrupted while acquiring or releasing a lock, causing other threads to spin wait for this one to run again.  
When a thread cannot acquire the lock, we are careful to add the current thread to the queue.  
The flag does not get set back to 0 when another thread gets woken up. When a thread is woken up, it will be returning from park, but it does not hold the guard at that point in the code and thus cannot try to set the flag to 1. We pass the thread to the next thread and set the flag during this transition.   
Wakeup/waiting race: There could be a race conditino before the call to park. With the wrong timing, a thread will be about to park, and a switch to another thread could lead to trouble, if the thread then released the lock. The park by the first thread would sleep forever.  
This can be solved by using setpark(), when the thread can indicate when it is about to park. If it is interrupted, and another thread calles unpark, the subsequent park returns immediately.

### 28.15 Different OS, Different Support, not necessary, some stuff about futex

### 28.16 2 phase locks

2 phase locks has 2 phases, one phase in which the lock spins for a while, hoping that it can acquire a lock. If the lock is not acquired in the first phase, a second phase is entered, where the caller is put to sleep and then woken up when the lock is available.